<a href="https://colab.research.google.com/github/milanoscookie/code/blob/master/analysis/listennojudge_anal_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import polars as pl
from scipy import signal, stats
from plotly import express as plx
import json
import csv

from sklearn import linear_model

In [ ]:
# !. ../secrets.sh && cd '../build' && ./scout_exe

In [5]:
COINBASE_TICKERS = '../data/coinbase_tickers.txt'
COINBASE_BOOKS = '../data/coinbase_books.txt'

KRAKEN_TICKERS = '../data/kraken_tickers.txt'
KRAKEN_BOOKS = '../data/kraken_books.txt'


DEPTH = 100
SCALE = int(1e8)

Explore the lag within the books as they update much faster than tickers do

In [7]:
# coinbase

cb_book = pd.read_csv(COINBASE_BOOKS, header=None)
cb_book.columns = ['ticker', 'timestamp', 'type', 'price', 'quantity']
cb_bids = cb_book[cb_book.type == 'bid'].drop('type',axis=1)
cb_asks = cb_book[cb_book.type == 'offer'].drop('type',axis=1)

# kraken

kk_book = pd.read_csv(KRAKEN_BOOKS, header=None)
kk_book.columns = ['ticker', 'timestamp', 'type', 'price', 'quantity']
kk_book = kk_book[kk_book['timestamp'] != 0]
kk_bids = kk_book[kk_book.type == 'bid'].drop('type',axis=1)
kk_asks = kk_book[kk_book.type == 'ask'].drop('type',axis=1)


print("size of cb_bids", len(cb_bids))
print("size of cb_asks", len(cb_asks))
print("size of kk_bids", len(kk_bids))
print("size of kk_asks", len(kk_asks))

signal_ind = {'cb_bids': 0, 'cb_asks': 1, 'kk_bids': 2, 'kk_asks': 3}
books = [cb_bids, cb_asks, kk_bids, kk_asks]

FileNotFoundError: [Errno 2] No such file or directory: '../data/coinbase_books.txt'

In [ ]:
# use to test code so it doesn't take forver to run
for sig,ii in signal_ind.items():
    books[ii] = books[ii].iloc[0:2000,:]
books

In [ ]:
best_signal = [pd.DataFrame(columns=['timestamp', 'price', 'quantity']) for x in signal_ind.keys()]
wavg_signal = [pd.DataFrame(columns=['timestamp', 'price', 'total_volume']) for x in signal_ind.keys()]
price_signal = [pd.DataFrame(columns=['timestamp', 'std', 'median', 'spread']) for x in signal_ind.keys()]

for sig, ii in signal_ind.items():
    is_bid = ii % 2 == 0
    allts = np.sort(books[ii]['timestamp'].unique())
    firstts = allts[0]
    at_firstts = books[ii]['timestamp'] == firstts

    curbook = books[ii][at_firstts].copy()
    # complete_book = pd.DataFrame()
    # this can probs easily be solved by using a groupby, im so dumb
    for ts in allts:
        at_ts = books[ii]['timestamp'] == ts
        sig_at_ts = books[ii][at_ts]

        merged = pd.merge(sig_at_ts[['price', 'quantity']], curbook, on='price', how='left', suffixes=('_old', ''))
        merged['quantity']=merged['quantity'].fillna(merged['quantity_old'])
        curbook = merged.drop('quantity_old', axis=1)
        curbook['timestamp'] = ts
        curbook = curbook[curbook['quantity'] != 0]
        curbook = curbook.sort_values('price', ascending=(not is_bid))

        # complete_book = pd.concat([complete_book,curbook.iloc[:DEPTH]])

        if(not curbook['price'].empty):
            minbook = curbook.iloc[0:DEPTH, :]
            wavg_signal[ii].loc[len(wavg_signal[ii])] = [ts,
                                                         (minbook['price'] * minbook['quantity']).sum() / minbook['quantity'].sum(),
                                                         minbook['quantity'].sum()]
            best_signal[ii].loc[len(best_signal[ii])] = [ts, curbook['price'].iloc[0], curbook['quantity'].iloc[0]]
            price_signal[ii].loc[len(price_signal[ii])] = [ts,
                                                           np.std(minbook['price']),
                                                           minbook['price'].iloc[int(len(minbook['price'])/2)],
                                                           np.abs(np.min(minbook['price']) - np.max(minbook['price']))]


    wavg_signal[ii] = wavg_signal[ii][wavg_signal[ii]['price'] > 0]
# best_signal
# wavg_signal
# price_signal

In [ ]:
def ffill_df(dfs, sigkeys):
    ffilled_dfs = []
    for sig,df in zip(sigkeys,dfs):
        df.index = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.drop(['timestamp'], axis=1)
        df.index.name = 'date'
        df = df.resample('1ms').ffill()
        ffilled_dfs.append(df)


    i = 0
    ffilled = ffilled_dfs[i].rename(columns=lambda col: col + "_" + sigkeys[i] if col != "date" else col)
    i += 1
    while i < len(sigkeys):
        current_df = ffilled_dfs[i].rename(columns=lambda col: col + "_" + sigkeys[i] if col != "date" else col)
        ffilled = pd.merge(
            ffilled,
            current_df,
            on='date',
            how='inner'
        )
        i += 1

    ffilled['timestamp'] = ffilled.index.view('int64') / 10**9
    ffilled['timestamp'] = ffilled['timestamp'] - np.min(ffilled['timestamp'])
    return ffilled


best_signal_ffilled = ffill_df(best_signal, list(signal_ind.keys()))
wavg_signal_ffilled = ffill_df(wavg_signal, list(signal_ind.keys()))
price_signal_ffilled = ffill_df(price_signal, list(signal_ind.keys()))

best_signal_ffilled.to_csv('best_signal_ffilled.csv')
wavg_signal_ffilled.to_csv('wavg_signal_ffilled.csv')
price_signal_ffilled.to_csv('price_signal_ffilled.csv')

In [ ]:
best_signal_ffilled = pd.read_csv('best_signal_ffilled.csv')
wavg_signal_ffilled = pd.read_csv('wavg_signal_ffilled.csv')
price_signal_ffilled = pd.read_csv('price_signal_ffilled.csv')

In [ ]:
## TODO: EVERYTHING ABOVE THIS NEEDS TO BE OPTIMIZED CUZ RN IT CAN'T RUN!!!

In [6]:
from google.colab import drive
drive.mount('/content/drive')

best_signal_ffilled = pd.read_csv('drive/MyDrive/signals forward filled/best_signal_ffilled.csv')
wavg_signal_ffilled = pd.read_csv('drive/MyDrive/signals forward filled/wavg_signal_ffilled.csv')
price_signal_ffilled = pd.read_csv('drive/MyDrive/signals forward filled/price_signal_ffilled.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# TODO: indicators

 best-ask to best-bid price spread
 ask-bid imbalance (ask_vol - bid_vol) / (ask_vol + bid_vol)

##### for MA, do for both best (best_signal_ffilled) and weighted avg (wavg_signal_ffileed)
- MA (3 ms)
- MA (10 ms)
- MA (100 ms)
- MA (1000 ms)
- EMA (alpha = 0.01)
- EMA (alpha = 0.05)
- EMA (alpha = 0.10)
- EMA (alpha = 0.33)
- EMA (alpha = 0.67)
- EMA (alpha = 0.90)

- kalman filter

*also I have a few ideas for slope interpolation of weighted avg to patch in gaps in noisy best_price, but we will ignore for now*

# Model pseudo code

next, code up logistic regression to tell us if we should trade or not
`LOGISTIC(indicators) => y(t)`
or `LINEAR(indicators) => x(t), calculate z scores => y(t)`

    x(T) = statistically segnificant signals at T timestep(forward filled)
    x(T) is statsitically segniciant at T if X(T) > MEAN + Z*STD  
    where MEAN & STD come from X(T-1), X(T-2), ... X(T-N)

Y = `COINBASE_BEST_ASK(T+(LAG))` < the `KRAKEN_BEST_BID within (T+(LAG), T+(LAG)+(HOLDTIME)]`

then optimize `HOLDTIME` and `Z` & `N` with the objective function (i'm thinking gridsearch to keep it simple)

    OBJ = 0
    WE_TRADE = Y(T) >= 0.5
    if(WE_TRADE):
      check if ask price @ T+LAG > bid price T1 @ T+LAG to T+LAG+HOLDTIME:
          if so, (OBJ += (ASK_PRICE(T+LAG) - BID_PRICE(T+LAG+T1))
          if not, (OBJ += ASK_PRICE(T+LAG) - BID_PRICE(T+LAG+T1))

    MAX OBJ W.R.T `HOLDTIME` and `Z` & `N`




In [36]:
# ID-ing signiciant signals, ideally use logistical regression for this part
window_size = 1000  # law of large numbers, not enough; make it like 1 s so like 1000
# TODO, not price, its like cb_ask_price
X1 = best_signal_ffilled['price_cb_asks']

mu = X1.rolling(window_size).mean().dropna()
sigma = X1.rolling(window_size).std().dropna()
z_th = 1
zscores = (X1 - mu)/sigma
zscores = zscores

sig = zscores > z_th # peaks
peaks = wavg_signal_ffilled['price_cb_asks'][sig]

del window_size, X1, mu, sigma, z_th, zscores, sig

peaks

,price_cb_asks
1781,4.955566e+07
1782,4.955566e+07
1783,4.955566e+07
1784,4.955566e+07
1785,4.955566e+07
...,...
10796176,4.846109e+07
10796177,4.846109e+07
10796178,4.846109e+07
10796179,4.846109e+07


In [40]:
HOLDTIME = 200 # ms
AFTERPEAK_HOLD = 10 # ms
LAG = 100 # 100 ms

THRESHOLD = 0.005*1e8
def find_first_peak(window):
    # for i, value in enumerate(window):
    #     if value > THRESHOLD:
    #         return i
    # return np.nan
    return window[(window > THRESHOLD).idxmax()]

result_df = pd.DataFrame()
result_df['initial_price'] = best_signal_ffilled['price_cb_bids'].iloc[:-(HOLDTIME+LAG)]
result_df['peak_price'] = best_signal_ffilled['price_kk_asks'].rolling(HOLDTIME + LAG, min_periods=(HOLDTIME+LAG)).max()
first_high_ind = (best_signal_ffilled['price_kk_asks'] - best_signal_ffilled['price_cb_bids']).rolling(HOLDTIME + LAG, min_periods=(HOLDTIME+LAG)).apply(find_first_peak)
#result_df['after_hold_first_high'] = best_signal_ffilled['price_kk_asks'].iloc[first_high_ind + AFTERPEAK_HOLD]
#result_df['first_high'] = best_signal_ffilled['price_kk_asks'].iloc[first_high_ind]
#result_df = result_df.dropna()
first_high_ind


/usr/local/lib/python3.10/dist-packages/pandas/core/series.py:1031: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


IndexError: positional indexers are out-of-bounds

In [45]:
np.sort(first_high_ind.dropna().unique())

array([-1.0000000e+04,  0.0000000e+00,  1.0000000e+04,  2.0000000e+04,
        3.0000000e+04,  4.0000000e+04,  5.0000000e+04,  6.0000000e+04,
        7.0000000e+04,  8.0000000e+04,  9.0000000e+04,  1.0000000e+05,
        1.1000000e+05,  1.2000000e+05,  1.3000000e+05,  1.4000000e+05,
        1.5000000e+05,  1.6000000e+05,  1.7000000e+05,  1.8000000e+05,
        1.9000000e+05,  2.0000000e+05,  2.1000000e+05,  2.2000000e+05,
        2.3000000e+05,  2.4000000e+05,  2.5000000e+05,  2.6000000e+05,
        2.7000000e+05,  2.8000000e+05,  2.9000000e+05,  3.0000000e+05,
        3.1000000e+05,  3.2000000e+05,  3.3000000e+05,  3.4000000e+05,
        3.5000000e+05,  3.6000000e+05,  3.7000000e+05,  3.8000000e+05,
        3.9000000e+05,  4.0000000e+05,  4.1000000e+05,  4.2000000e+05,
        4.3000000e+05,  4.4000000e+05,  4.5000000e+05,  4.6000000e+05,
        4.7000000e+05,  4.8000000e+05,  4.9000000e+05,  5.0000000e+05,
        5.1000000e+05,  5.2000000e+05,  5.3000000e+05,  5.4000000e+05,
      

In [ ]:
result_df['final_price'] = best_signal_ffilled['price_kk_asks'].iloc[HOLDTIME+LAG:]
result_df['trade_success'] = result_df['after_hold_first_high'] > (result_df['initial_price'] + THRESHOLD)

result_df['sale_price'] = result_df.apply(lambda row: row['first_high'] if row['trade_success'] else row['final_price'], axis=1)
result_df['profit'] = result_df['sale_price'] - result_df['initial_price']



In [ ]:
total_profit = result_df['profit'][peaks.index].sum()
win_ratio = result_df['trade_success'][peaks.index].sum() / len(peaks.index)

sharpe = result_df['return'].mean() / result_df['return'].std()

(total_profit, win_ratio, sharpe)

Make the following two cells into a method with parameters `HOLDTIME` and

In [ ]:
from matplotlib import pyplot as plt
plt.plot(best_signal_ffilled['price_cb_asks'])
plt.plot(best_signal_ffilled['price_cb_bids'])
plt.ylim(0,1)

In [ ]:
# plot ask lag
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=best_signal_ffilled['timestamp'], y=best_signal_ffilled['price_' + "cb" + "_asks"]/SCALE, mode='lines', name='Coinbase'))
fig.add_trace(go.Scatter(x=best_signal_ffilled['timestamp'], y=best_signal_ffilled['price_' + "kk" + "_asks"]/SCALE, mode='lines', name='Kraken'))
fig.show()

In [ ]:
# plot bids
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=best_signal_ffilled['timestamp'], y=best_signal_ffilled['price_' + "cb" + "_bids"]/SCALE, mode='lines', name='Coinbase'))
fig.add_trace(go.Scatter(x=best_signal_ffilled['timestamp'], y=best_signal_ffilled['price_' + "kk" + "_bids"]/SCALE, mode='lines', name='Kraken'))
fig.show()

In [ ]:

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=midpt_signal['timestamp'], y=midpt_signal['midpt_' + "cb"]/SCALE, mode='lines', name='Coinbase'))
fig.add_trace(go.Scatter(x=midpt_signal['timestamp'], y=midpt_signal['midpt_' + "kk"]/SCALE, mode='lines', name='Kraken'))
fig.show()

In [ ]:
def max_delayed_crosscorrelation(s1, s2):
    correlation = signal.correlate(s1, s2, mode='full', method='auto')

    max_corr_index = np.argmax(correlation)
    max_corr_value = correlation[max_corr_index]

    delay = max_corr_index - (len(s1) - 1)

    return max_corr_value, delay

In [ ]:
print('asks', max_delayed_crosscorrelation(best_signal_ffilled['price_cb_asks'], best_signal_ffilled['price_kk_asks']))
print('bids', max_delayed_crosscorrelation(best_signal_ffilled['price_cb_bids'], best_signal_ffilled['price_kk_bids']))
print('midpt', max_delayed_crosscorrelation(midpt_signal['midpt_cb'], midpt_signal['midpt_kk']))